In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def make_dataloader(n, batch_size):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=batch_size)
    return data_loader

In [23]:
import torch.nn as nn
from metal.mmtl.task import Task

BATCHSIZE = 32

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo_data = make_dataloader(5000, batch_size=BATCHSIZE)
bar_data = make_dataloader(2000, batch_size=BATCHSIZE)

foo = Task("foo_task", {"train": foo_data, "valid": foo_data}, foo_input, foo_head)
bar = Task("bar_task", {"train": bar_data, "valid": bar_data}, bar_input, bar_head)
# baz = Task("baz_task", "baz_head", [make_dataloader(100), None, None])
tasks = [foo, bar]

In [24]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks, device=-1, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model, 
    tasks, 
    n_epochs=3, 
    lr=0.1, 
    progress_bar=True,
    log_unit="batches",
    log_every=1,
    score_every=1,
    grad_clip=0.01,
    writer=None,
    checkpoint_best=True,
    checkpoint_metric="foo_task/valid/accuracy",
    checkpoint_metric_mode="max",
)

Beginning train loop.
Expecting a total of _approximately_ 5600 examples and 175 batches per epoch from 2 tasks.


[ (0.01 epo)]: TRAIN:[loss=0.713] VALID:[foo_task/accuracy=0.660, bar_task/accuracy=0.659]
Saving model at iteration 1 with best (max) score 0.660
[ (0.01 epo)]: TRAIN:[loss=0.640] VALID:[foo_task/accuracy=0.656, bar_task/accuracy=0.799]
[ (0.02 epo)]: TRAIN:[loss=0.452] VALID:[foo_task/accuracy=0.764, bar_task/accuracy=0.804]
Saving model at iteration 3 with best (max) score 0.764
[ (0.02 epo)]: TRAIN:[loss=0.359] VALID:[foo_task/accuracy=0.884, bar_task/accuracy=0.876]
Saving model at iteration 4 with best (max) score 0.884
[ (0.03 epo)]: TRAIN:[loss=0.363] VALID:[foo_task/accuracy=0.964, bar_task/accuracy=0.926]
Saving model at iteration 5 with best (max) score 0.964
[ (0.03 epo)]: TRAIN:[loss=0.240] VALID:[foo_task/accuracy=0.910, bar_task/accuracy=0.960]
[ (0.04 epo)]: TRAIN:[loss=0.126] VALID:[foo_task/accuracy=0.885, bar_task/accuracy=0.946]
[ (0.05 epo)]: TRAIN:[loss=0.124] VALID:[foo_task/accuracy=0.877, bar_task/accuracy=0.949]
[ (0.05 epo)]: TRAIN:[loss=0.165] VALID:[foo_tas

[ (0.50 epo)]: TRAIN:[loss=0.005] VALID:[foo_task/accuracy=0.991, bar_task/accuracy=0.993]
[ (0.50 epo)]: TRAIN:[loss=0.001] VALID:[foo_task/accuracy=0.991, bar_task/accuracy=0.993]
[ (0.51 epo)]: TRAIN:[loss=0.041] VALID:[foo_task/accuracy=0.994, bar_task/accuracy=0.994]
[ (0.51 epo)]: TRAIN:[loss=0.020] VALID:[foo_task/accuracy=0.994, bar_task/accuracy=0.994]
[ (0.52 epo)]: TRAIN:[loss=0.032] VALID:[foo_task/accuracy=0.995, bar_task/accuracy=0.993]
[ (0.53 epo)]: TRAIN:[loss=0.005] VALID:[foo_task/accuracy=0.997, bar_task/accuracy=0.988]
Saving model at iteration 92 with best (max) score 0.997
[ (0.53 epo)]: TRAIN:[loss=0.000] VALID:[foo_task/accuracy=0.993, bar_task/accuracy=0.984]
[ (0.54 epo)]: TRAIN:[loss=0.001] VALID:[foo_task/accuracy=0.988, bar_task/accuracy=0.983]
[ (0.54 epo)]: TRAIN:[loss=0.008] VALID:[foo_task/accuracy=0.988, bar_task/accuracy=0.982]
[ (0.55 epo)]: TRAIN:[loss=0.025] VALID:[foo_task/accuracy=0.990, bar_task/accuracy=0.983]
[ (0.55 epo)]: TRAIN:[loss=0.008]

[ (1.01 epo)]: TRAIN:[loss=0.000] VALID:[foo_task/accuracy=0.990, bar_task/accuracy=0.989]
[ (1.01 epo)]: TRAIN:[loss=0.002] VALID:[foo_task/accuracy=0.995, bar_task/accuracy=0.993]
[ (1.02 epo)]: TRAIN:[loss=0.024] VALID:[foo_task/accuracy=0.994, bar_task/accuracy=0.994]
[ (1.02 epo)]: TRAIN:[loss=0.012] VALID:[foo_task/accuracy=0.994, bar_task/accuracy=0.996]
[ (1.03 epo)]: TRAIN:[loss=0.014] VALID:[foo_task/accuracy=0.989, bar_task/accuracy=0.996]
[ (1.03 epo)]: TRAIN:[loss=0.024] VALID:[foo_task/accuracy=0.987, bar_task/accuracy=0.995]
[ (1.04 epo)]: TRAIN:[loss=0.052] VALID:[foo_task/accuracy=0.989, bar_task/accuracy=0.997]
[ (1.05 epo)]: TRAIN:[loss=0.016] VALID:[foo_task/accuracy=0.986, bar_task/accuracy=0.995]
[ (1.05 epo)]: TRAIN:[loss=0.037] VALID:[foo_task/accuracy=0.986, bar_task/accuracy=0.994]
[ (1.06 epo)]: TRAIN:[loss=0.001] VALID:[foo_task/accuracy=0.985, bar_task/accuracy=0.994]
[ (1.06 epo)]: TRAIN:[loss=0.005] VALID:[foo_task/accuracy=0.984, bar_task/accuracy=0.994]

[ (1.54 epo)]: TRAIN:[loss=0.036] VALID:[foo_task/accuracy=0.992, bar_task/accuracy=0.993]
[ (1.54 epo)]: TRAIN:[loss=0.001] VALID:[foo_task/accuracy=0.995, bar_task/accuracy=0.996]
[ (1.55 epo)]: TRAIN:[loss=0.034] VALID:[foo_task/accuracy=0.994, bar_task/accuracy=0.997]
[ (1.55 epo)]: TRAIN:[loss=0.002] VALID:[foo_task/accuracy=0.987, bar_task/accuracy=0.986]
[ (1.56 epo)]: TRAIN:[loss=0.115] VALID:[foo_task/accuracy=0.983, bar_task/accuracy=0.981]
[ (1.57 epo)]: TRAIN:[loss=0.087] VALID:[foo_task/accuracy=0.982, bar_task/accuracy=0.979]
[ (1.57 epo)]: TRAIN:[loss=0.002] VALID:[foo_task/accuracy=0.983, bar_task/accuracy=0.979]
[ (1.58 epo)]: TRAIN:[loss=0.269] VALID:[foo_task/accuracy=0.984, bar_task/accuracy=0.981]
[ (1.58 epo)]: TRAIN:[loss=0.131] VALID:[foo_task/accuracy=0.987, bar_task/accuracy=0.983]
[ (1.59 epo)]: TRAIN:[loss=0.000] VALID:[foo_task/accuracy=0.990, bar_task/accuracy=0.987]
[ (1.59 epo)]: TRAIN:[loss=0.000] VALID:[foo_task/accuracy=0.991, bar_task/accuracy=0.991]

[ (2.01 epo)]: TRAIN:[loss=0.011] VALID:[foo_task/accuracy=0.974, bar_task/accuracy=0.981]
[ (2.01 epo)]: TRAIN:[loss=0.000] VALID:[foo_task/accuracy=0.978, bar_task/accuracy=0.982]
[ (2.02 epo)]: TRAIN:[loss=0.000] VALID:[foo_task/accuracy=0.981, bar_task/accuracy=0.983]
[ (2.02 epo)]: TRAIN:[loss=0.000] VALID:[foo_task/accuracy=0.982, bar_task/accuracy=0.985]
[ (2.03 epo)]: TRAIN:[loss=0.435] VALID:[foo_task/accuracy=0.987, bar_task/accuracy=0.985]
[ (2.03 epo)]: TRAIN:[loss=0.102] VALID:[foo_task/accuracy=0.991, bar_task/accuracy=0.991]
[ (2.04 epo)]: TRAIN:[loss=0.001] VALID:[foo_task/accuracy=0.997, bar_task/accuracy=0.995]
[ (2.05 epo)]: TRAIN:[loss=0.001] VALID:[foo_task/accuracy=0.997, bar_task/accuracy=0.997]
[ (2.05 epo)]: TRAIN:[loss=0.000] VALID:[foo_task/accuracy=0.994, bar_task/accuracy=0.996]
[ (2.06 epo)]: TRAIN:[loss=0.015] VALID:[foo_task/accuracy=0.991, bar_task/accuracy=0.994]
[ (2.06 epo)]: TRAIN:[loss=0.000] VALID:[foo_task/accuracy=0.989, bar_task/accuracy=0.992]

[ (2.54 epo)]: TRAIN:[loss=0.000] VALID:[foo_task/accuracy=0.993, bar_task/accuracy=0.992]
[ (2.54 epo)]: TRAIN:[loss=0.001] VALID:[foo_task/accuracy=0.985, bar_task/accuracy=0.989]
[ (2.55 epo)]: TRAIN:[loss=0.278] VALID:[foo_task/accuracy=0.981, bar_task/accuracy=0.983]
[ (2.55 epo)]: TRAIN:[loss=0.020] VALID:[foo_task/accuracy=0.979, bar_task/accuracy=0.981]
[ (2.56 epo)]: TRAIN:[loss=0.000] VALID:[foo_task/accuracy=0.977, bar_task/accuracy=0.978]
[ (2.57 epo)]: TRAIN:[loss=0.000] VALID:[foo_task/accuracy=0.975, bar_task/accuracy=0.977]
[ (2.57 epo)]: TRAIN:[loss=0.046] VALID:[foo_task/accuracy=0.973, bar_task/accuracy=0.976]
[ (2.58 epo)]: TRAIN:[loss=0.177] VALID:[foo_task/accuracy=0.974, bar_task/accuracy=0.976]
[ (2.58 epo)]: TRAIN:[loss=0.219] VALID:[foo_task/accuracy=0.975, bar_task/accuracy=0.974]
[ (2.59 epo)]: TRAIN:[loss=0.521] VALID:[foo_task/accuracy=0.979, bar_task/accuracy=0.976]
[ (2.59 epo)]: TRAIN:[loss=0.320] VALID:[foo_task/accuracy=0.984, bar_task/accuracy=0.981]

In [ ]:
for batch in foo.data_loaders["train"]:
    X, Y = batch
    print(model(X, ['foo_task']))
    print(model.calculate_loss(X, Y, ['foo_task']))    
    print(model.calculate_output(X, ['foo_task']))    
    break

In [ ]:
next(model.parameters()).device